# TD1 — La Modélisation Dimensionnelle (Schéma en Étoile)

**Durée estimée** : 1h30  
**Niveau** : Débutant  
**Objectifs pédagogiques** :
1.  **Concevoir** un schéma en étoile à partir d'un besoin métier (Ventes Retail).
2.  **Implémenter** les tables de dimensions et de faits en SQL.
3.  **Comprendre** les concepts fondamentaux :
    *   **Grain** : Quelle est la finesse de l'information ?
    *   **Dimensions** : Les axes d'analyse (Qui ? Quoi ? Quand ? Où ?).
    *   **Fait** : La mesure chiffrée (Combien ?).
    *   **Clés de Substitution (Surrogate Keys)** : Pourquoi ne pas utiliser les IDs opérationnels ?

---

In [ ]:
# Import SQLite (exécution minimale)
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
print('Base TD1 créée (SQLite en mémoire)')

Base TD1 créée


## Partie 1 : Le Modèle Conceptuel (Mermaid)

Nous allons modéliser un processus de vente simple.

### Les choix de modélisation :
-   **Fait** : `FACT_VENTES` contenant les métriques (`montant`, `quantite`).
-   **Dimensions** :
    -   `DIM_DATE` : Pour l'analyse temporel (Année, Mois, Trimestre).
    -   `DIM_PRODUIT` : Pour savoir ce qui est vendu (Catégorie, Nom).
    -   `DIM_MAGASIN` : Pour la localisation géographique (Ville, Région).

### Le Diagramme Entité-Relation :
```mermaid
erDiagram
  DIM_DATE ||--o{ FACT_VENTES : date_id
  DIM_PRODUIT ||--o{ FACT_VENTES : produit_sk
  DIM_MAGASIN ||--o{ FACT_VENTES : magasin_sk

  DIM_DATE {
    int date_id PK "Clé technique"
    date date_cal "Date réelle"
    int annee
    int mois
    int trimestre
  }
  DIM_PRODUIT {
    int produit_sk PK "Clé de substitution"
    string produit_id "Code produit (Source)"
    string produit_nom
    string categorie
  }
  DIM_MAGASIN {
    int magasin_sk PK "Clé de substitution"
    string magasin_id "Code magasin (Source)"
    string ville
    string region
  }
  FACT_VENTES {
    int fact_sk PK
    int date_id FK
    int produit_sk FK
    int magasin_sk FK
    int quantite "Mesure additive"
    decimal montant "Mesure additive"
  }
```

<!-- Cellule de transition -->
Dans cette partie, nous allons créer les tables.
Nous utilisons des `INTEGER PRIMARY KEY AUTOINCREMENT` pour gérer automatiquement les clés de substitution (Surrogate Keys).

### Concepts Clés : Grain et Clés

1.  **Le Grain (Granularity)** :
    -   C'est la définition d'une ligne unique dans la table de faits.
    -   *Ici* : **Une ligne = Une vente d'un produit spécifique, dans un magasin spécifique, à une date donnée.**
    -   C'est le niveau de détail le plus fin.

2.  **Clés de Substitution (Surrogate Keys - SK)** :
    -   Remarquez `produit_sk` (entier) vs `produit_id` (code source 'P01').
    -   **Pourquoi ?**
        -   **Indépendance** : Si le code produit change dans le système source, notre historique reste intact.
        -   **Performance** : Les jointures sur des entiers (`INT`) sont plus rapides que sur des chaînes de caractères (`VARCHAR`).
        -   **Historisation** : Permet de gérer plusieurs versions d'un même produit (SCD Type 2).

In [ ]:
# Partie 2 : Implémentation SQL (DDL)

cursor.execute('''
-- Dimension Temps : Granularité journalière
CREATE TABLE dim_date (
  date_id INTEGER PRIMARY KEY AUTOINCREMENT,
  date_cal DATE NOT NULL,
  annee INTEGER,
  mois INTEGER,
  jour INTEGER,
  trimestre INTEGER
);
''')

cursor.execute('''
-- Dimension Produit : Avec clé de substitution (SK)
CREATE TABLE dim_produit (
  produit_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  produit_id TEXT UNIQUE, -- Code source (Business Key)
  produit_nom TEXT,
  categorie TEXT
);
''')

cursor.execute('''
-- Dimension Magasin
CREATE TABLE dim_magasin (
  magasin_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  magasin_id TEXT UNIQUE, -- Code source
  ville TEXT,
  region TEXT
);
''')

cursor.execute('''
-- Table de Faits : Contient les clés étrangères et les mesures
CREATE TABLE fact_ventes (
  fact_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  date_id INTEGER NOT NULL,
  produit_sk INTEGER NOT NULL,
  magasin_sk INTEGER NOT NULL,
  montant REAL,   -- Mesure
  quantite INTEGER, -- Mesure
  -- Contraintes d'intégrité référentielle
  FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
  FOREIGN KEY (produit_sk) REFERENCES dim_produit(produit_sk),
  FOREIGN KEY (magasin_sk) REFERENCES dim_magasin(magasin_sk)
);
''')
print('✅ Tables créées avec succès.')

Tables créées


<!-- Cellule de transition -->
Une fois la structure en place, nous devons l'alimenter.
Dans un projet réel, cette étape est réalisée par l'outil ETL (Extract-Transform-Load).
Ici, nous simulons le chargement avec des instructions `INSERT`.

In [ ]:
# Partie 3 : Chargement des Données (Simulation ETL)

# 1. Chargement des Dimensions
# Note : Dans la réalité, on ferait un "Lookup" pour vérifier si l'entrée existe déjà.

cursor.executemany('''
INSERT INTO dim_date (date_cal, annee, mois, jour, trimestre) VALUES (?, ?, ?, ?, ?);
''', [
    ('2024-01-02', 2024, 1, 2, 1),
    ('2024-01-03', 2024, 1, 3, 1),
    ('2024-01-04', 2024, 1, 4, 1)
])

cursor.executemany('''
INSERT INTO dim_produit (produit_id, produit_nom, categorie) VALUES (?, ?, ?);
''', [
    ('P01', 'Chemise Oxford', 'Textile'),
    ('P02', 'Sneakers Run', 'Chaussure'),
    ('P03', 'Jeans Slim', 'Textile')
])

cursor.executemany('''
INSERT INTO dim_magasin (magasin_id, ville, region) VALUES (?, ?, ?);
''', [
    ('M01', 'Paris', 'IDF'),
    ('M02', 'Lyon', 'ARA')
])

# 2. Chargement de la Table de Faits
# Note : Ici on utilise les ID générés (1, 2, 3...) directement pour simplifier.
# En production, l'ETL chercherait le produit_sk correspondant au produit_id 'P01'.

cursor.executemany('''
INSERT INTO fact_ventes (date_id, produit_sk, magasin_sk, montant, quantite) VALUES (?, ?, ?, ?, ?);
''', [
    (1, 1, 1, 120.0, 2), -- 2 Chemises à Paris le 02/01
    (1, 2, 1, 75.0, 1),  -- 1 Sneakers à Paris le 02/01
    (2, 1, 2, 60.0, 1),  -- 1 Chemise à Lyon le 03/01
    (3, 3, 1, 90.0, 1)   -- 1 Jean à Paris le 04/01
])
print('✅ Données chargées (Dimensions + Faits).')

Données insérées


<!-- Cellule de transition -->
Le modèle est chargé. Il faut maintenant vérifier qu'il répond aux questions métier.
Nous allons exécuter 3 requêtes types d'analyse décisionnelle.

In [ ]:
# Partie 4 : Validation du Modèle

# 1. Volume total
# Vérifier que nous avons bien nos 4 transactions.
cursor.execute('SELECT COUNT(*) AS nb_lignes FROM fact_ventes;')
print(f'Volume total : {cursor.fetchone()[0]} lignes (Attendu: 4)')

# 2. Top Produits par Chiffre d'Affaires
# C'est une requête typique de BI : Agrégation par dimension.
print('\n🏆 Top Produits par CA :')
cursor.execute('''
SELECT 
    p.produit_nom, 
    SUM(f.montant) AS ca_total
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
GROUP BY p.produit_nom
ORDER BY ca_total DESC
LIMIT 3;
''')
for row in cursor.fetchall():
    print(f"- {row[0]} : {row[1]} €")

# 3. Analyse Croisée : CA par Ville et par Mois
# On joint 2 dimensions (Temps et Magasin) à la table de faits.
print('\n🌍 CA par Ville et Mois :')
cursor.execute('''
SELECT 
    d.annee, 
    d.mois, 
    m.ville, 
    SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY d.annee, d.mois, m.ville
ORDER BY d.annee, d.mois, m.ville;
''')
for row in cursor.fetchall():
    print(f"[{row[0]}-{row[1]:02d}] {row[2]} : {row[3]} €")

conn.close()
print('\n✅ TD1 terminé avec succès.')

Volume total : 4
Top 3 produits :
('Chemise Oxford', 180.0)
('Jeans Slim', 90.0)
('Sneakers Run', 75.0)
CA par ville/mois :
(2024, 1, 'Lyon', 60.0)
(2024, 1, 'Paris', 285.0)
TD1 terminé
